In [1]:
import pandas as pd
import numpy as np

This is just for SIDS at the moment. Can be updated for all countries when we have the data

In [2]:
#Names to WB names
any_to_wb = pd.read_csv("inputs/any_name_to_wb_name.csv",index_col="any",squeeze=True)

#GAR names with SIDS spec
gar_name_sids = pd.read_csv("inputs/gar_name_sids.csv")

#iso3 to wb country name table
iso3_to_wb=pd.read_csv("inputs/iso3_to_wb_name.csv").set_index("iso3").squeeze()

#iso2 to iso3 table
iso2_iso3 = pd.read_csv("inputs/names_to_iso.csv", usecols=["iso2","iso3"]).drop_duplicates().set_index("iso2").squeeze() #the tables has more lines than countries to account for several ways of writing country names

In [3]:
path = "sids data\\"

## Roads -- km from OSM and hard coding road costs

In [210]:
roads = pd.read_csv(path+"km_roads_sids.csv")

In [211]:
roads[['Total Length (km)', 'Total Primary',\
       'Total Secondary', 'Total Tertiary', 'Residential', 'Track / service',\
       'Total Other']] = roads[['Total Length (km)', 'Total Primary',\
       'Total Secondary', 'Total Tertiary', 'Residential', 'Track / service',\
       'Total Other']].astype(float)

In [212]:
roads["country"] = roads.Country.replace(any_to_wb)

In [213]:
#cost per km of road based on st Lucia data. 
primary_road_cost = 1200000
secondary_road_cost = 700000
tertiary_road_cost = 600000

In [214]:
#reduced road values because of depreciation.
lifetime_primary = 20
lifetime_secondary = 15
lifetime_tertiary = 10

In [215]:
roads = roads.fillna(0)
roads = roads.set_index("country")

In [216]:
roads["average_age_primary"] = 8
roads["average_age_secondary"] = 5
roads["average_age_tertiary"] = 5

In [217]:
roads.loc["Kiribati","average_age_primary"]=12

In [218]:
(1-1/lifetime_primary*roads.average_age_primary)

country
Cuba                   0.6
Dominican Republic     0.6
Haiti                  0.6
Papua New Guinea       0.6
Jamaica                0.6
Fiji                   0.6
Belize                 0.6
Bahamas, The           0.6
Mauritius              0.6
Barbados               0.6
Samoa                  0.6
Antigua and Barbuda    0.6
Maldives               0.6
Bermuda                0.6
St Lucia               0.6
Kiribati               0.4
Name: average_age_primary, dtype: float64

In [219]:
roads["total_roads_value"] = roads["Total Primary"]*primary_road_cost*(1-1/lifetime_primary*roads.average_age_primary)+\
                             roads["Total Secondary"]*secondary_road_cost*(1-1/lifetime_secondary*roads.average_age_secondary)+\
                             roads["Total Tertiary"]*tertiary_road_cost*(1-1/lifetime_tertiary*roads.average_age_tertiary)+\
                             roads["Residential"]*tertiary_road_cost*(1-1/lifetime_tertiary*roads.average_age_tertiary)+\
                             roads["Track / service"]*tertiary_road_cost*(1-1/lifetime_tertiary*roads.average_age_tertiary)+\
                             roads["Total Other"]*tertiary_road_cost*(1-1/lifetime_tertiary*roads.average_age_tertiary)

In [220]:
roads.sample(5)

,Country,Sheet,Total Length (km),Total Primary,Total Secondary,Total Tertiary,Residential,Track / service,Total Other,average_age_primary,average_age_secondary,average_age_tertiary,total_roads_value
country,,,,,,,,,,,,,
Haiti,Haiti,HTI_UTM18N,30127.365970,1035.937743,1616.294096,1414.995709,5088.878595,9988.709733,10982.550100,8,5,5,9.742686e+09
"Bahamas, The",Bahamas,BHS_UTM18N,7033.335154,679.961233,512.171878,445.811236,2699.568179,1752.007260,943.815368,8,5,5,2.480946e+09
Antigua and Barbuda,Antigua and Barabuda,ATG_UTM20N,1368.449906,104.578230,70.561461,54.123172,757.917842,379.964498,1.304702,8,5,5,4.662181e+08
Mauritius,Mauritius,MUS_UTM40N,5534.874399,250.030055,807.943827,369.117497,2724.279486,798.350166,585.153368,8,5,5,1.900132e+09
Jamaica,Jamaica,JAM_UTM18N,14969.821590,1069.891646,1072.979872,1255.614165,7609.429350,1510.734627,2451.171930,8,5,5,5.119131e+09


## Power (from Platts)

In [95]:
power = pd.read_csv(path+"power_plants_sids.csv")
power["country"] = power.COUNTRY.apply(lambda x:x.title()).replace(any_to_wb)

In [96]:
power["total_power_value"] = power.CAPEX

## Buildings data from Rashmin and PCRAFI 

In [128]:
buildings_pcrafi = pd.read_csv(path+"PCRAFI_stocks.csv")
#values in million USD

In [129]:
#we divide by 3 because for Fiji PCRAFI is 3 times higher than Rashmin's assessment, which is consistent with GAR
buildings_pcrafi = 1/3*10**6*buildings_pcrafi.set_index("country")

In [130]:
buildings_pcrafi.rename(columns={"building":"total_building_value"},inplace=True)

In [131]:
buildings_pcrafi

,total_building_value,infrastructure
country,,
Cook Islands,4.322667e+08,3.920000e+07
Fiji,6.288400e+09,1.031300e+09
"Micronesia, Fed. Sts.",5.763333e+08,1.042667e+08
Kiribati,3.353667e+08,5.473333e+07
Marshall Islands,4.680333e+08,9.530000e+07
Nauru,5.793333e+07,2.466667e+07
Niue,1.368667e+08,1.400000e+07
Papua New Guinea,1.316967e+10,2.213033e+09
Palau,4.461667e+08,5.330000e+07


In [132]:
buildings_rashmin = pd.read_csv(path+"Gross_Capital_Summary.csv")
buildings_rashmin["country"] = buildings_rashmin.country.replace(iso3_to_wb)

In [133]:
buildings_rashmin["total_building_value"] = buildings_rashmin["Buildings (Residential + Non-Res)"]

In [134]:
buildings_rashmin.set_index("country").total_building_value

country
Belize                4.620789e+09
St. Lucia             3.197446e+09
Jamaica               3.822092e+10
Haiti                 2.656431e+10
Grenada               2.131954e+09
Dominican Republic    1.530000e+11
Fiji                  6.504790e+09
FJI est 2                      NaN
Name: total_building_value, dtype: float64

In [135]:
#Fiji has 2 estimates. first one is rashmin, last one is PCRAFI
buildings = pd.concat([buildings_rashmin.set_index("country"),buildings_pcrafi]).reset_index().drop_duplicates("country",keep="first")

## Water from WASH infrastructure needs

In [163]:
water = pd.read_csv(path+"water_infra_value_sids.csv")

In [164]:
water.set_index("country").sample()

,total_water_value
country,
Vanuatu,37989860.6


## Merge all available infra 

In [222]:
all_infra = pd.concat([roads.total_roads_value,power.set_index("country").total_power_value,\
                       buildings.set_index("country").total_building_value,water.set_index("country")],axis=1)

In [223]:
all_infra.dropna()

,total_roads_value,total_power_value,total_building_value,total_water_value
Belize,2.513105e+09,246783703.0,4.620789e+09,4.352581e+08
Fiji,2.981647e+09,483010130.0,6.504790e+09,4.135154e+08
Haiti,9.742686e+09,223696330.0,2.656431e+10,1.271568e+09
Jamaica,5.119131e+09,787424726.0,3.822092e+10,4.261407e+09
Kiribati,3.843400e+08,4982963.0,3.353667e+08,1.082715e+07
Papua New Guinea,6.976123e+09,889191070.0,1.316967e+10,8.536102e+08
Samoa,9.025303e+08,47664858.0,7.159667e+08,7.177621e+07


## Read total capital data gathered in "gather_capital_data" and wb data gathered in "download_wb_data"

In [224]:
K = pd.read_csv("intermediate/avg_prod_k_with_gar_for_sids.csv",index_col="Unnamed: 0")
wb = pd.read_csv("inputs/wb_data.csv",index_col="country")

## recomputes Ktot from WB GDP and average capital productivity

In [225]:
wb["Ktot"] = wb.gdp_pc_pp*wb['pop']/K.avg_prod_k

In [226]:
sids_infra = pd.concat([all_infra,wb.Ktot],axis=1).dropna()
sids_infra

,total_roads_value,total_power_value,total_building_value,total_water_value,Ktot
Belize,2.513105e+09,246783703.0,4.620789e+09,4.352581e+08,1.069437e+10
Fiji,2.981647e+09,483010130.0,6.504790e+09,4.135154e+08,2.244152e+10
Haiti,9.742686e+09,223696330.0,2.656431e+10,1.271568e+09,1.325637e+11
Jamaica,5.119131e+09,787424726.0,3.822092e+10,4.261407e+09,1.146077e+11
Kiribati,3.843400e+08,4982963.0,3.353667e+08,1.082715e+07,7.417660e+08
Papua New Guinea,6.976123e+09,889191070.0,1.316967e+10,8.536102e+08,6.381417e+10


In [227]:
sids_infra["transport_share"] = sids_infra.total_roads_value/sids_infra.Ktot
sids_infra["energy_share"] = sids_infra.total_power_value/sids_infra.Ktot
sids_infra["building_share"] = sids_infra.total_building_value/sids_infra.Ktot
sids_infra["water_share"] = sids_infra.total_water_value/sids_infra.Ktot

In [228]:
sids_infra

,total_roads_value,total_power_value,total_building_value,total_water_value,Ktot,transport_share,energy_share,building_share,water_share
Belize,2.513105e+09,246783703.0,4.620789e+09,4.352581e+08,1.069437e+10,0.234993,0.023076,0.432077,0.040700
Fiji,2.981647e+09,483010130.0,6.504790e+09,4.135154e+08,2.244152e+10,0.132863,0.021523,0.289855,0.018426
Haiti,9.742686e+09,223696330.0,2.656431e+10,1.271568e+09,1.325637e+11,0.073494,0.001687,0.200389,0.009592
Jamaica,5.119131e+09,787424726.0,3.822092e+10,4.261407e+09,1.146077e+11,0.044667,0.006871,0.333493,0.037183
Kiribati,3.843400e+08,4982963.0,3.353667e+08,1.082715e+07,7.417660e+08,0.518142,0.006718,0.452119,0.014596
Papua New Guinea,6.976123e+09,889191070.0,1.316967e+10,8.536102e+08,6.381417e+10,0.109319,0.013934,0.206375,0.013376


In [229]:
sids_infra[["transport_share","energy_share","building_share","water_share"]].sum(axis=1)

Belize              0.730846
Fiji                0.462668
Haiti               0.285163
Jamaica             0.422213
Kiribati            0.991575
Papua New Guinea    0.343005
dtype: float64

### Residential vs non residential buildings

In [154]:
## hard coding Fiji data from Rashmin's file Fiji_2015_exposure_estimation_110517.xlsx
fiji_total = 4093521135+2411268610
fiji_residential = 4093521135/fiji_total

# for the US, according to the BEA, residential is 19,199.1 billion and non residential 21,879.4 billion. so by default we put 50/50.

In [176]:
sids_infra.loc["Fiji","building_residential_share"] = fiji_residential
sids_infra["building_residential_share"] = sids_infra.building_residential_share.fillna(0.5)

In [177]:
sids_infra

,total_roads_value,total_power_value,total_building_value,total_water_value,Ktot,transport_share,energy_share,building_share,water_share,building_residential_share
Belize,2.447768e+09,246783703.0,4.620789e+09,4.352581e+08,1.069437e+10,0.228884,0.023076,0.432077,0.040700,0.500000
Fiji,2.921762e+09,483010130.0,6.504790e+09,4.135154e+08,2.244152e+10,0.130194,0.021523,0.289855,0.018426,0.629309
Haiti,9.618373e+09,223696330.0,2.656431e+10,1.271568e+09,1.325637e+11,0.072557,0.001687,0.200389,0.009592,0.500000
Jamaica,4.990744e+09,787424726.0,3.822092e+10,4.261407e+09,1.146077e+11,0.043546,0.006871,0.333493,0.037183,0.500000
Kiribati,3.925000e+08,4982963.0,3.353667e+08,1.082715e+07,7.417660e+08,0.529143,0.006718,0.452119,0.014596,0.500000
Papua New Guinea,6.749944e+09,889191070.0,1.316967e+10,8.536102e+08,6.381417e+10,0.105775,0.013934,0.206375,0.013376,0.500000
